# Beginner Ray: Embarrassingly Parallel Workloads using Ray Core

[Ray](https://docs.ray.io/en/latest/index.html) has a rich ecosystem of libraries and frameworks for Machine Learning.
These are built on top of the simple primitives provided by [Ray Core](https://docs.ray.io/en/latest/ray-core/walkthrough.html).
This notebook will walk through a complete example using **remote tasks** in Ray Core, to cover the following concepts:
* Setting up your Ray Cluster in Domino
* Connecting to your Ray Cluster in Domino
* Submitting remote Tasks using the ray.remote decorator and retrieving results
* Interpreting logs and errors for basic troubleshooting
* Effectively distributing tasks in parallel while avoiding common [anti-patterns](https://docs.ray.io/en/latest/ray-core/tasks/patterns/index.html)
* Bonus: Ray Cluster auto-scaling in Domino

This tutorial is designed to serve two purposes:
1. Provide a conceptual introduction to Ray, which will serve as a foundation for the Intermediate tutorials and further explorations with Ray.
2. Function as a practical example for writing custom distributed code using only Ray Core, which can be applied directly to certain types of repeated simulations.

## Sample problem: Monte Carlo Approximation of Pi

First, let's introduce our simple example problem: approximating the value of Pi using a Monte Carlo approach. Wikipedia has a good explanation and visualization [here](https://en.wikipedia.org/wiki/Monte_Carlo_method#Overview). The procedure is as follows:
* Generate a series of random points in a 1x1 square (x and y between 0 and 1).
* Calculate for each point whether it falls inside a radius of 1.
* Multiply the ratio of the number of points falling **inside** the unit radius and the **total** number of points to approximiate Pi.

Below is a code snippet to do this, with no dependencies outside a random number generator and the time library to benchmark the computation time. Try experimenting with the sample size and see how it impacts the accuracy of the approximation and the time required.

In [ ]:
import random

def monte_carlo_pi_sampler(n_samples, debug=False):
    n_inside_quadrant = 0
    if debug:
        print(f"monte_carlo_pi_sampler getting ready to do {n_samples} samples")
    for _ in range(n_samples):
        x = random.uniform(0,1)
        y = random.uniform(0,1)
        r = (x**2 + y**2)**0.5
        if r <= 1:
            n_inside_quadrant += 1
    if debug:
        print(f"monte_carlo_pi_sampler found {n_inside_quadrant} inside a unit circle")
    return n_inside_quadrant

In [ ]:
import time

start = time.time()

# Try experimenting with different sample sizes, and turning on debug mode
n_total = 10**6
n_inside = monte_carlo_pi_sampler(n_total)
pi = 4*n_inside/n_total

time_elapsed = time.time() - start
print(f'Pi is approximately {pi}.')
print(f'It took {time_elapsed:.2f}s with {n_total} total samples')

## Setting up and connecting to your Ray cluster

You should already have setup a workspace as recommended in the `README.md`, which includes the required setup for the Ray cluster.
Remember that the following properties are set as part of the Workspace Settings in Domino:
* Whether there is a Ray cluster attached at all
* How many workers the cluster starts with, and whether it can auto-scale by adding more workers
* What hardware tier the cluster uses for head and worker nodes
* What software environment the cluster uses
* Whether additional local storage is attached (not relevant in this tutorial)

See the README for more details about the recommended settings.

Inside the workspace, you should have a tab for the **Ray Web UI**.
Duplicate your workspace tab so you can see the Ray Web UI side-by-side with this notebook as you run the rest of the tutorial. 

### Connecting to a "local" Ray cluster (the wrong way)

Below is some typical code for initializing a Ray cluster.
Beware of the fact that this will NOT connect you to your Domino on-demand cluster!
Instead, it will create a new Ray cluster that lives only on the "local" workspace node.
It is easy to be fooled into thinking that you are correctly connected to your Ray cluster after doing this, especially if using a large Hardware Tier, because Ray can still effectively **parallelize** work on a single multi-core machine.
However, if you were to run the remainder of this notebook after initializing Ray like this, you could see in the Ray Web UI that nothing is happening - your work would not be **distributed** to the multiple nodes of the Domino cluster.

In [ ]:
## Don't initialize Ray this way on Domino!

import ray
if ray.is_initialized() == False:
    ray.init()

One way to prove that Ray is not correctly connected to your Domino cluster is by inspecting the list of nodes.
The first thing you may notice is that there is only one node listed, whereas we expect 3 nodes (1 head node and 2 worker nodes).
If you look closer at the IP address of the node, you can also see that it does not match any of the IP addresses in the Ray Web UI.

In [ ]:
ray.nodes()

### Disconnecting from and shutting down your Ray cluster

Notice the `if ray.is_initialized()` statement above, which will prevent errors about Ray already being running if you try to initialize it again.
This also means that we need to shut down our connection to this cluster before we can reconnect the "correct" way below.

In [ ]:
ray.shutdown()

This can also be used to reset the connection to the Domino Ray cluster.
Note that in that case it only shuts down the **processes** running on the Ray nodes, not the head or worker nodes themselves; to completely restart the cluster, stop and start the Workspace.

There are a few cases where you might run Ray as a "local cluster" like this deliberately even in Domino.
* To test some code locally as a baseline when troubleshooting issues with your Ray cluster, especially suspected mismatches between workspace and cluster environments.
* To use Ray on a single large machine on earlier versions of Domino that don't support On-Demand Ray clusters, or any other scenario when you are unable to launch a workspace with a Ray cluster attached.

### Connecting to your Ray cluster (the right way)

Domino provides the information needed to connect to your Ray cluster via environment variables.
After initializing Ray this way, you should see subsequent operations in Ray represented in the Ray Web UI.

In [ ]:
import ray
import os

if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")

Inspecting the nodes again, you should also see that the IP addresses listed now match what you see in the Ray Web UI.

In [ ]:
ray.nodes()

## Submitting work to the Ray cluster

### Tasks (with ray.remote), futures, and results

Now that we have initialized Ray, we can submit work to the cluster by defining a function using the `ray.remote` decorator.
Wrapping our Pi example this way is extremely simple:

In [ ]:
@ray.remote
def ray_monte_carlo_pi_sampler(n_samples, debug=False):
    return monte_carlo_pi_sampler(n_samples, debug=debug)

To submit this to the cluster as a Task, call the function using `.remote`. Some important things to remember:
* When you call a remote function, it returns a `future` instead of the actual result.
* Ray will start executing the function immediately - Ray tasks do NOT use "lazy execution".
* To get the actual result, call `.get` on the future.

In [ ]:
n_total = 10**6
n_inside_future = ray_monte_carlo_pi_sampler.remote(n_total)
print('So far, we have only submitted our function call and gotten a future back:')
n_inside_future

In [ ]:
n_inside = ray.get(n_inside_future)
print(f'Now we have the final result:')
n_inside

In [ ]:
pi = 4*n_inside/n_total
print(f'Pi is approximately {pi}.')

Note that `.get` will be a **blocking** call, meaning it will wait for the task to finish. On the other hand, `.remote` is **non-blocking** because it immediately returns a future. Let's run the above sequence in a single cell with timing to demonstrate:

In [ ]:
start = time.time()

n_total = 10**7
n_inside_future = ray_monte_carlo_pi_sampler.remote(n_total)

print(f'Submitting took {time.time()-start:.2f}s')

n_inside = ray.get(n_inside_future)
print(f'The full calculation took {time.time()-start:.2f}s with {n_total} total samples')

pi = 4*n_inside/n_total
print(f'Pi is approximately {pi}.')

If you are watching the Ray Web UI in parallel to running this notebook, you should see evidence that this function is running on the cluster!
However, all we've done so far is move the work from our local machine to the head node.
Nothing is running in parallel, and all the work is being done on a single cluster node.
We'll fix that in the next section, but first we'll take a very important detour into basic troubleshooting tips.

### Logs, errors, and PIDs

You may have noticed the `debug` option we have been ignoring in the `monte_carlo_pi_sampler` function. Now that you have the basics of futures and blocking vs non-blocking calls, let's use that option to generate some print statements from inside the Ray tasks. Here is what the debug output looks like without Ray:

In [ ]:
monte_carlo_pi_sampler(10**6, debug=True)

Now let's run the Ray version in debug mode. Try running the following two cells two ways:
* First, wait a few seconds between them
* Second, run them back-to-back

Notice the **PID** of the Ray processes the print statements come from, and compare them to what you see in the Ray Web UI.

In [ ]:
n_total = 10**6
n_inside_future = ray_monte_carlo_pi_sampler.remote(n_total, debug=True)

In [ ]:
n_inside = ray.get(n_inside_future)
pi = 4*n_inside/n_total
print(f'Pi is approximately {pi}.')

Notice how it will print out to your current cell's output whenever Ray gets to that point in the calculation, regardless of what cells you have run in the meantime.
In other words, the print statements from Ray tasks can come in a **different order** compared to print statements in the local notebook depending on how you run the cells!
In addition, there is just enough "lag" for printing from remote tasks that you may find the final Pi result displaying before the final `ray_monte_carlo_pi_sampler` printout, even though the final Pi result must clearly come only after all remote tasks are complete.

Now let's trigger an obvious error and see what happens. First, try it without Ray.

In [ ]:
monte_carlo_pi_sampler('a million', debug=True)

Next, submit a Ray task with the same bad input and see what happens. You'll notice our first debug line print out, but nothing after that...

In [ ]:
n_inside_future = ray_monte_carlo_pi_sampler.remote('a million', debug=True)

Finally, try to get the result. Now we see the error message! Unlike the print statements, the error trace does not appear "as soon as it happens" - we have to try to get the result first. Notice how the stack trace includes the full "path" of the task through the Ray code, and the meaningful part of the error is really the last few lines.

Don't spend too much time staring at the details here - just keep in mind this section of the tutorial as a reference in case you are ever debugging remote functions in a "real" scenario. Comparing the structure of this "known" error to errors you encounter in the wild may help with interpreting and troubleshooting.

In [ ]:
n_inside = ray.get(n_inside_future)

## Parallelize and distribute the work

The Monte Carlo approximation of Pi is a good example of an **Embarassingly parallel** problem, because it is very easy to break it up into independent chunks for parallel processing. However, there are some nuances and pitfalls to this process.

### The importance of batching

At one extreme, we have the code in our previous section - technically we are using Ray, but we are getting no benefits because we have not actually broken up the computation. Nothing is parallelized at all. At the other extreme, we could consider sending each random point generation (and radius calculation) as a separate task to the cluster. This would incur unnecessary overhead, as explained in the [Ray docs](https://docs.ray.io/en/latest/ray-core/tasks/patterns/fine-grained-tasks.html).

Instead, we should use our existing code to calculate **batches** of points. At this point, you may notice a clever thing in the original code: `monte_carlo_pi_sampler` simply returns a count of how many points fall inside the unit radius, and the actual calculation of Pi happens outside the function. This is deliberate, to make it easier to split the calculation up into batches like below. If you are adapting some existing code to make use of Ray, you will often need to do some restructuring to get a similar structure.

First, let's just write and test a new "batching" function with no parallel or distributed processing as a baseline. We'll include all the timing and print statements inside this function for convenience.

In [ ]:
def approximate_pi(sample_batch_sizes, debug=False):
    start = time.time()
    total_samples = sum(sample_batch_sizes)
    total_inside = 0
    for i,n in enumerate(sample_batch_sizes):
        total_inside += monte_carlo_pi_sampler(n, debug=debug)
        print(f"Time check after batch {i+1}: {time.time()-start:.2f}s")
    pi = 4*total_inside/total_samples
    print(f'Pi is approximately {pi}.')
    print(f'It took {time.time()-start:.2f}s with batches: {sample_batch_sizes}')

In [ ]:
approximate_pi(3*[10**7])

### First try at distributing (the wrong way)
Now modify our batching function and try to distribute each batch to cluster workers!

In [ ]:
def ray_approximate_pi_wrong(sample_batch_sizes, debug=False):
    start = time.time()
    total_samples = sum(sample_batch_sizes)
    total_inside = 0
    for i,n in enumerate(sample_batch_sizes):
        n_inside_future = ray_monte_carlo_pi_sampler.remote(n, debug=debug)
        total_inside += ray.get(n_inside_future)
        print(f"Time check after batch {i+1}: {time.time()-start:.2f}s")
    pi = 4*total_inside/total_samples
    print(f'Pi is approximately {pi}.')
    print(f'It took {time.time()-start:.2f}s with batches: {sample_batch_sizes}')

In [ ]:
ray_approximate_pi_wrong(3*[10**7], debug=True)

Can you figure out what we are doing wrong? Why isn't it running in parallel? It is still only using the head node! (The answer is a common [antipattern](https://docs.ray.io/en/latest/ray-core/tasks/patterns/ray-get-loop.html)/pitfall for beginners.)

### Second try at distributing (the right way)

To actually parallelize the work, it's important to submit all our batches first before attempting to get any results. The below function does this the "correct" way, and is the first time in this notebook we submit any tasks to the Ray **worker** nodes (as opposed to just the **head** node) - hooray!

In [ ]:
def ray_approximate_pi(sample_batch_sizes, debug=False):
    start = time.time()
    total_samples = sum(sample_batch_sizes)
    n_inside_futures = []
    for i,n in enumerate(sample_batch_sizes):
        n_inside_futures.append(ray_monte_carlo_pi_sampler.remote(n, debug=debug))
        print(f'Time check after batch {i+1} submit: {time.time()-start:.2f}s')
    total_inside = 0
    for j, future in enumerate(n_inside_futures):
        total_inside += ray.get(future)
        print(f'Time check after batch {j+1} result: {time.time()-start:.2f}s')
    pi = 4*total_inside/total_samples
    print(f'Pi is approximately {pi}.')
    print(f'It took {time.time()-start:.2f}s with batches: {sample_batch_sizes}')

In [ ]:
ray_approximate_pi(3*[10**7], debug=True)

Since this is the first time we are submitting Tasks to the worker nodes, this is the first time you will see any worker processes with **PIDs** (with each worker identified by **ip address**) in the Ray Web UI. These processes will take a second or so to startup, so you may notice a second or two difference in the timing of the previous cell the first time you run it. This is to be expected! Once the worker processes are started, they will remain until/unless you shutdown your Ray connection.

### Alternate ways to collect results
Ray can handle a list of futures just fine, so we can make our code a little more concise with no problems.
For very small batches, you may also notice an improvement in performance getting results this way, compared to the previous section.

In [ ]:
def ray_approximate_pi_alt(sample_batch_sizes, debug=False):
    start = time.time()
    total_samples = sum(sample_batch_sizes)
    n_inside_futures = []
    for i,n in enumerate(sample_batch_sizes):
        n_inside_futures.append(ray_monte_carlo_pi_sampler.remote(n, debug=debug))
        print(f"Time check after batch {i+1} submit: {time.time()-start:.2f}s")
    total_inside = sum(ray.get(n_inside_futures))
    pi = 4*total_inside/total_samples
    print(f'Pi is approximately {pi}.')
    print(f'It took {time.time()-start:.2f}s with batches: {sample_batch_sizes}')

In [ ]:
ray_approximate_pi_alt(3*[10**7])

In fact, the previous section example of calling `ray.get` one by one in order is actually [another antipattern](https://docs.ray.io/en/latest/ray-core/tasks/patterns/submission-order.html). In our case, it doesn't really matter because we have to wait for all tasks to be finished before doing the remaining computations anyway. (And besides, if you use the default batches provided, they are all the same size and so are expected to take the same amount of time to compute.) However, real problems will not always be so tidy, and using `ray.wait` as suggested by the above link will return tasks as they finish no matter the order of submission. Check out the Intermediate tutorial for an example using `ray.wait`!

## Putting it all together, with bonus autoscaling demo

Now that we've walked through the important concepts for getting started on Ray, let's put all the code together in one place, and do a bit of a "stress test".

Domino On-Demand clusters have the (optional) ability to **autoscale**, meaning they can add workers when needed.
Restart your workspace as follows to enable autoscaling:
* Find the Workspace **Settings** in the left navigation pane.
* Click **Edit Settings**.
* Go to Step 3 for **Compute Cluster** settings.
* Check the box to **Auto-scale workers**, and enter **4** for the max. (Min should be at 2.)
* Click **Save and Restart**.

By default, all On-Demand clusters (including Ray) will scale up when the average CPU utilization is 80%, and will scale back down when it is below the threshold for 5 minutes.
This is [configurable](https://docs.dominodatalab.com/en/latest/admin_guide/71d6ad/central-configuration/#compute-cluster-auto-scaling) by the Domino Admin, who can also choose to disable autoscaling for any cluster type.

Once the workspace has restarted, you can start at this section of the notebook - no need to rerun previous cells.

In [ ]:
import random
import time
import ray
import os

In [ ]:
# Make sure to connect to the Domino Ray cluster correctly!

if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")

In [ ]:
# Here is the main workhorse function for our problem, written "before Ray".

def monte_carlo_pi_sampler(n_samples, debug=False):
    n_inside_quadrant = 0
    if debug:
        print(f"monte_carlo_pi_sampler getting ready to do {n_samples} samples")
    for _ in range(n_samples):
        x = random.uniform(0,1)
        y = random.uniform(0,1)
        r = (x**2 + y**2)**0.5
        if r <= 1:
            n_inside_quadrant += 1
    if debug:
        print(f"monte_carlo_pi_sampler found {n_inside_quadrant} inside a unit circle")
    return n_inside_quadrant

In [ ]:
# Now we wrap it with the ray.remote decorator!

@ray.remote
def monte_carlo_pi_sampler_on_ray(n_samples, debug=False):
    return monte_carlo_pi_sampler(n_samples, debug=debug)

In [ ]:
# And now we write the code to submit tasks to the Ray cluster

def approximate_pi_on_ray(sample_batch_sizes, debug=False):
    start = time.time()
    total_samples = sum(sample_batch_sizes)
    n_inside_futures = []
    for i,n in enumerate(sample_batch_sizes):
        n_inside_futures.append(monte_carlo_pi_sampler_on_ray.remote(n, debug=debug))
        print(f"Time check after batch {i+1} submit: {time.time()-start:.2f}s")
    total_inside = sum(ray.get(n_inside_futures))
    pi = 4*total_inside/total_samples
    print(f'Pi is approximately {pi}.')
    print(f'It took {time.time()-start:.2f}s with batches: {sample_batch_sizes}')

In [ ]:
# Kick off some small batches to make sure everything is working
approximate_pi_on_ray(3*[10**6])

In [ ]:
# Finally, try more and larger batches here to trigger the cluster to autoscale!
approximate_pi_on_ray(10*[10**8], debug=True)

### Congratulations!

You have finished this Beginner tutorial on Ray Core, where we covered the following:
* Setting up your Ray Cluster in Domino
* Connecting to your Ray Cluster in Domino
* Submitting remote Tasks using the ray.remote decorator and retrieving results
* Interpreting logs and errors for basic troubleshooting
* Effectively distributing tasks in parallel while avoiding common [anti-patterns](https://docs.ray.io/en/latest/ray-core/tasks/patterns/index.html)
* Bonus: Ray Cluster auto-scaling in Domino

### What's next?

The example in this tutorial is essentially a **simulation**, and one that can easily be broken up into smaller pieces.
In other words, it is **Embarrassingly Parallel**.
It requires passing very little data to each Task, because as a simulation it generates most of the data within the Task itself.
The results to pass back from the Task are likewise very small.
In a more realistic simulation, you will almost certainly be generating that data with Numpy arrays or similar constructs to take advantage of vectorized operations, rather than looping through simluated points one-by-one, but as long as this is done within the Task it doesn't change the basic structure of the problem.

If your use-case happens to match this pattern, you can now use Ray to distribute your computations!

If not, check out the Intermediate tutorials for more about the following topics:
* Handling **large data**, passing it efficiently around the cluster when needed and avoiding unnecessary data transfer
* Using **more Ray Core functionality**, like Actors and ray.wait
* Integrating with **additional libraries**, like XGBoost and Ray Tune